In [1]:
import pandas as pd


In [2]:


# df_one_hot_regions = pd.get_dummies(df['WHO Region'], prefix='Region')


# df = pd.concat([df, df_one_hot_regions], axis=1)


# df.shape

In [3]:
#https://www.kaggle.com/datasets/georgesaavedra/covid19-dataset
covid_data = pd.read_csv("covid19_dataset/covid-data.csv")

#https://www.kaggle.com/datasets/gpreda/covid-world-vaccination-progress
vaccines_data= pd.read_csv("covid19_dataset/country_vaccinations.csv")

#https://www.kaggle.com/datasets/iamsouravbanerjee/world-population-dataset
world_statistics =pd.read_csv("covid19_dataset/world_population.csv")




In [4]:
#creating a dataframe that contains data about covid cases and deaths per date and country
columns_that_I_want = ["date","iso_code","location","continent","new_cases","total_cases","new_deaths","total_deaths"]
covid_data = covid_data[columns_that_I_want]

In [5]:
#creating a dataframe that contains data about vaccination per date and country
vaccines_data = vaccines_data[["date", "iso_code","total_vaccinations","people_vaccinated","people_fully_vaccinated"]]

In [6]:
#mergin my dataframes based on their country code and date
first_merge_df = pd.merge(covid_data, vaccines_data, on=['iso_code',"date"], how='left')

In [7]:
first_merge_df

,date,iso_code,location,continent,new_cases,total_cases,new_deaths,total_deaths,total_vaccinations,people_vaccinated,people_fully_vaccinated
0,2020-02-24,AFG,Afghanistan,Asia,5.0,5.0,NaN,NaN,NaN,NaN,NaN
1,2020-02-25,AFG,Afghanistan,Asia,0.0,5.0,NaN,NaN,NaN,NaN,NaN
2,2020-02-26,AFG,Afghanistan,Asia,0.0,5.0,NaN,NaN,NaN,NaN,NaN
3,2020-02-27,AFG,Afghanistan,Asia,0.0,5.0,NaN,NaN,NaN,NaN,NaN
4,2020-02-28,AFG,Afghanistan,Asia,0.0,5.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
166321,2022-03-01,ZWE,Zimbabwe,Africa,491.0,236871.0,0.0,5395.0,7901360.0,4365856.0,3399915.0
166322,2022-03-02,ZWE,Zimbabwe,Africa,632.0,237503.0,1.0,5396.0,7910740.0,4368726.0,3402434.0
166323,2022-03-03,ZWE,Zimbabwe,Africa,0.0,237503.0,0.0,5396.0,7921113.0,4372925.0,3406482.0
166324,2022-03-04,ZWE,Zimbabwe,Africa,1236.0,238739.0,1.0,5397.0,7930621.0,4374896.0,3408609.0


In [8]:

columns_that_I_want = ["CCA3", "2020 Population"]
world_statistics = world_statistics[columns_that_I_want]
world_statistics


,CCA3,2020 Population
0,AFG,38972230
1,ALB,2866849
2,DZA,43451666
3,ASM,46189
4,AND,77700
...,...,...
229,WLF,11655
230,ESH,556048
231,YEM,32284046
232,ZMB,18927715


In [9]:
second_merge_df = pd.merge(first_merge_df, world_statistics, left_on=['iso_code'], right_on=['CCA3'],how='inner')

columns_to_drop = ['CCA3']

# Drop the specified columns
second_merge_df = second_merge_df.drop(columns=["CCA3"])

In [10]:
second_merge_df

,date,iso_code,location,continent,new_cases,total_cases,new_deaths,total_deaths,total_vaccinations,people_vaccinated,people_fully_vaccinated,2020 Population
0,2020-02-24,AFG,Afghanistan,Asia,5.0,5.0,NaN,NaN,NaN,NaN,NaN,38972230
1,2020-02-25,AFG,Afghanistan,Asia,0.0,5.0,NaN,NaN,NaN,NaN,NaN,38972230
2,2020-02-26,AFG,Afghanistan,Asia,0.0,5.0,NaN,NaN,NaN,NaN,NaN,38972230
3,2020-02-27,AFG,Afghanistan,Asia,0.0,5.0,NaN,NaN,NaN,NaN,NaN,38972230
4,2020-02-28,AFG,Afghanistan,Asia,0.0,5.0,NaN,NaN,NaN,NaN,NaN,38972230
...,...,...,...,...,...,...,...,...,...,...,...,...
153993,2022-03-01,ZWE,Zimbabwe,Africa,491.0,236871.0,0.0,5395.0,7901360.0,4365856.0,3399915.0,15669666
153994,2022-03-02,ZWE,Zimbabwe,Africa,632.0,237503.0,1.0,5396.0,7910740.0,4368726.0,3402434.0,15669666
153995,2022-03-03,ZWE,Zimbabwe,Africa,0.0,237503.0,0.0,5396.0,7921113.0,4372925.0,3406482.0,15669666
153996,2022-03-04,ZWE,Zimbabwe,Africa,1236.0,238739.0,1.0,5397.0,7930621.0,4374896.0,3408609.0,15669666


In [12]:
df_per_country = second_merge_df['iso_code'] !=  second_merge_df['iso_code'].shift()

# Forward-fill the specified column only within each country
merged_df['total_vaccinations'] = merged_df.groupby(df_per_country.cumsum())['total_vaccinations'].ffill()
merged_df['people_vaccinated'] = merged_df.groupby(df_per_country.cumsum())['people_vaccinated'].ffill()
merged_df['people_fully_vaccinated'] = merged_df.groupby(df_per_country.cumsum())['people_fully_vaccinated'].ffill()

NameError: name 'merged_df' is not defined

In [ ]:
merged_df_no_null_values = merged_df.fillna(0)


In [ ]:
merged_df_no_null_values.to_csv('vaccination_data_no_null.csv', index=False)

In [ ]:
filtered_df  = filtered_df.fillna(0)

In [ ]:
filtered_df

In [ ]:
filtered_df.to_csv('vaccination_data.csv', index=False)